# Lab 06 Stochastic Programming

Made & Presented by Bo Tang

In this lab, we will explore how Stochastic Programming is used to handle uncertainty in decision-making processes. We will focus on two-stage stochastic programming. We will formulate problem as extensive form and also show how Bender's decomposition to solve this.

In [1]:
# install gurobipy first
! pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 17.0 MB/s eta 0:00:00


In [2]:
import gurobipy as gp
import numpy as np
from gurobipy import GRB

### Stochastic Programming Review

Stochastic Programming is a powerful technique used to model optimization problems that involve uncertainty. Unlike deterministic models where all parameters are known with certainty, stochastic programming incorporates randomness in the data, making it highly useful in applications like finance, energy, and supply chain management where future outcomes are uncertain.


#### General Stochastic Program

Here is the general form of a stochastic program.

$$
\min_{\mathbf{x} \in \mathcal{X}} \mathbb{E} [ f(\mathbf{x}, \xi) ]
$$

- $\mathbf{x}$: decision variables with deterministic constraints $\mathcal{X}$.
- $\xi$: Random vector defined on some probability space $\Omega$.
- Expected value $\mathbb{E} [ f(\mathbf{x}, \xi) ] = \int_{\Omega} f(\mathbf{x}, \xi) d \mathbb{P}$.

The main goal is to minimize the expected cost or maximize the expected profit across all possible scenarios $\xi$.

#### Two-Stage Stochastic Program with Recourse

A common structure in stochastic programming is the two-stage model. In this approach, decisions are divided into:

1. First-stage decisions $\mathbf{x}$: Made before the uncertainty is revealed.
2. Second-stage (Recourse) decisions $\mathbf{y}$: Adjustments made after the uncertainty is observed.

The general form for a two-stage stochastic linear program can be written as:
$$
\min_{\mathbf{x} \in \mathcal{X}} \mathbf{c}^{\top} \mathbf{x} + \mathbb{E}_{\xi} \left[ Q(\mathbf{x}, \xi) \right]
$$
where recourse function:
$$
Q(\mathbf{x}, \xi) := \min_{\mathbf{y}} \mathbf{q}(\xi)^{\top} \mathbf{y}
$$
Subject to
$$
\mathbf{W}(\xi) \mathbf{y} = \mathbf{h}(\xi) - \mathbf{T}(\xi) \mathbf{x}
$$
$$
\mathbf{y} \geq \mathbf{0}
$$

$ \mathbf{q}(\xi), \mathbf{h}(\xi), \mathbf{W}(\xi), \mathbf{T}(\xi) $ are scenario-dependent parameters.

In practical, evaluating $\mathbb{E}_{\xi} \left[ Q(\mathbf{x}, \xi) \right]$ can be high-dimensional integral.

Question:

- What if $\mathbf{c}$ is random？
$$
\min_{\mathbf{x} \in \mathcal{X}} \mathbb{E}_{\xi} \left[ \mathbf{c}(\xi)^{\top} \mathbf{x} + Q(\mathbf{x}, \xi) \right]
$$
$$
= \min_{\mathbf{x} \in \mathcal{X}} \mathbb{E}_{\xi} \left[ \mathbf{c}(\xi)^{\top} \mathbf{x} \right] + \mathbb{E}_{\xi} \left[ Q(\mathbf{x}, \xi) \right]
$$
$$
= \min_{\mathbf{x} \in \mathcal{X}} \mathbb{E}_{\xi} \left[ \mathbf{c}(\xi) \right]^{\top} \mathbf{x} + \mathbb{E}_{\xi} \left[ Q(\mathbf{x}, \xi) \right]
$$
Hence, we can just take $\mathbb{E}_{\xi} \left[ \mathbf{c}(\xi) \right]$ as $\mathbf{c}$.

- Why cannot we use $Q \big( \mathbf{x}, \mathbb{E}_{\xi} [ \xi) ] \big)$ directly?

#### Extensive Form

When we have a **finite number of scenarios**, the **extensive form** includes all possible scenarios explicitly in a single model. For example, consider a two-stage problem with three scenarios, each having different probabilities $\mathcal{p}$. The extensive form combines the first-stage decision with recourse variables and constraints for each scenario.

$$
\min_{\mathbf{x}, \mathbf{y}} \mathbf{c}^{\top} \mathbf{x} + \sum_{\xi=1}^{\Xi} \mathcal{p}_{\xi} \mathbf{q}_{\xi}^{\top} \mathbf{y}_{\xi}
$$
Subject to
$$
\mathbf{x} \in \mathcal{X}
$$
$$
\mathbf{W}_{\xi} \mathbf{y}_{\xi} = \mathbf{h}_{\xi} - \mathbf{T}_{\xi} \mathbf{x}  \quad \forall {\xi} \in {\Xi}
$$
$$
\mathbf{y}_{\xi} \geq \mathbf{0} \quad \forall {\xi} \in {\Xi}
$$

#### Bender's Decomposition

While the extensive form captures the entire problem, it can quickly become computationally infeasible for problems with many scenarios due to the large number of variables and constraints. This is where decomposition methods come in.

$$
\min_{\mathbf{x} \in \mathcal{X}} \mathbf{c}^{\top} \mathbf{x} + \mathbb{E}_{\xi} \left[ Q(\mathbf{x}, \xi) \right]
$$
can be converted as
$$
\min_{\mathbf{x} \in \mathcal{X}, \eta} \mathbf{c}^{\top} \mathbf{x} + \sum_{\xi}^{\Xi} \mathcal{p}_{\xi} \eta_{\xi}
$$
Subject to
$$
\eta_{\xi} \geq Q_{\xi}(\mathbf{x})
$$

For $Q_{\xi}(\mathbf{x})$, the primal form is
$$
\min_{\mathbf{y}} \mathbf{q}_{\xi}^{\top} \mathbf{y}
$$
Subject to
$$
\mathbf{W}_{\xi} \mathbf{y} = \mathbf{h}_{\xi}- \mathbf{T}_{\xi} \mathbf{x}
$$
$$
\mathbf{y} \geq \mathbf{0}
$$
Therefore, we can obtain the duality:
$$
\max_{\boldsymbol{\pi}} \boldsymbol{\pi}^{\top} (\mathbf{h}_{\xi}- \mathbf{T}_{\xi} \mathbf{x})
$$
Subject to
$$
\mathbf{W}_{\xi}^\top \boldsymbol{\pi} \leq \mathbf{q}_{\xi}
$$

Let define $\Pi_{\xi} = \{\boldsymbol{\pi}: \mathbf{W}_{\xi}^\top \boldsymbol{\pi} \leq \mathbf{q}_{\xi} \}$ and $V(\Pi_{\xi})$ is the finite set of extreme points of $\Pi_{\xi}$.

We can observe:
$$
\eta_{\xi} \geq Q_{\xi}(\mathbf{x})
$$
$$
\Leftrightarrow \eta_{\xi} \geq \max_{\boldsymbol{\pi}} \{ \boldsymbol{\pi}^{\top} (\mathbf{h}_{\xi}- \mathbf{T}_{\xi} \mathbf{x}): \boldsymbol{\pi} \in \Pi_{\xi} \}
$$
$$
\Leftrightarrow \eta_{\xi} \geq \boldsymbol{\pi}^{\top} (\mathbf{h}_{\xi}- \mathbf{T}_{\xi} \mathbf{x}), \quad \forall \boldsymbol{\pi} \in V(\Pi_{\xi})
$$

Therefore, we have our **master problem** for Bender's:
$$
\min_{\mathbf{x}, \mathbf{y}} \mathbf{c}^{\top} \mathbf{x} + \sum_{\xi=1}^{\Xi} \mathcal{p}_{\xi} \eta_{\xi}
$$
Subject to
$$
\mathbf{x} \in \mathcal{X}
$$
$$
\mathbf{W}_{\xi} \mathbf{y}_{\xi} = \mathbf{h}_{\xi} - \mathbf{T}_{\xi} \mathbf{x}  \quad \forall {\xi} \in {\Xi}
$$
$$
\eta_{\xi} \geq \boldsymbol{\pi}_{\xi}^{\top} (\mathbf{h}_{\xi}- \mathbf{T}_{\xi} \mathbf{x}) \quad \forall \boldsymbol{\pi}_{\xi} \in V_{\xi}^t, \forall \xi \in \Xi
$$
where $V_{\xi}^t$ a (small) subset of $\Pi_{\xi}$ at iteraion $t$.

Let Let $\mathbf{x}^t, \eta_{\xi}^t$ be an optimal solution to master problem at iteraion $t$.
1. For each $\xi$, we need to solve subproblem $Q(\mathbf{x}_t, \xi)$ to obtain a $\boldsymbol{\pi}_{\xi}^t$ with the given $\mathbf{x}_t$.
2. If $\eta_{\xi} \geq Q(\mathbf{x}_t, \xi)$, all the constraints for scenario $\xi$ are satisfied. Do nothing.
3. If $\eta_{\xi} < Q(\mathbf{x}_t, \xi)$, add the constraint $\eta_{\xi} \geq {\boldsymbol{\pi}^t_{\xi}}^{\top} (\mathbf{h}_{\xi}- \mathbf{T}_{\xi} \mathbf{x})$ to master problem.

### Example from Leture Note

You own 500 acres of land in which to plant wheat (w), corn (c) and beets (b) with planting costs are 150 \$/acre for wheat, 230 \$/acre for corn and 260 \$/acre for beets.

To feed your cattle you require 200 T of wheat and 240 T of corn.
The yield of each crop is uncertain with expected values of 2.5, 3 and 20 T/acre for what, corn and beets respectively. Purchasing cost of wheat is 238 \$/T and 210 \$/T for corn. Any excess wheat can be sold for 170 \$/T, corn for 150 \$/ton, beets for 36 \$/ton (up to 6000 T), beets for 10 \$/T (beyond 6000 T).

Suppose that:
- There is a 1/3 probability that crops will be as expected
- There is a 1/3 probability that ALL crops will be 20% higher than expected
- There is a 1/3 probability that ALL crops will be 20% lower than expected.

In [ ]:
# data setup
planting_cost = {"wheat": 150, "corn": 230, "beets": 260}
purchase_cost = {"wheat": 238, "corn": 210}
selling_price = {"wheat": 170, "corn": 150, "beets_high": 36, "beets_low": 10}
feed_requirements = {"wheat": 200, "corn": 240}
land_available = 500
beet_sale_limit = 6000

In [ ]:
# scenarios and probabilities
expected_yields = {"wheat": 2.5, "corn": 3, "beets": 20}
scenarios = [1, 2, 3]
yields = {
    1: {crop: yield_val for crop, yield_val in expected_yields.items()},    # Scenario 1: Expected yields
    2: {crop: 1.2 * yield_val for crop, yield_val in expected_yields.items()},  # Scenario 2: 20% higher yields
    3: {crop: 0.8 * yield_val for crop, yield_val in expected_yields.items()}   # Scenario 3: 20% lower yields
}
probabilities = {1: 1/3, 2: 1/3, 3: 1/3}

In [ ]:
yields

{1: {'wheat': 2.5, 'corn': 3, 'beets': 20},
 2: {'wheat': 3.0, 'corn': 3.5999999999999996, 'beets': 24.0},
 3: {'wheat': 2.0, 'corn': 2.4000000000000004, 'beets': 16.0}}

#### Using Expected Yields

To get a handle on the incremental value of Stochastic Programming we will first solve the problem without SP, using a commonsense approach. This also gives us an opportunity to first develop the simpler deterministic LP.

Simpler approach
- Formulate the LP assuming expected yields.
- Solve the LP to determine the acreages.
- Evaluate the profit for each scenario.
- Calculate the expected profit.

First, let solve the LP with expected yields.

In [ ]:
# create Gurobi model
m = gp.Model("Simple Approach")

# decision variables
planting = m.addVars(planting_cost.keys(), vtype=GRB.CONTINUOUS, name="plants")
sales = m.addVars(selling_price.keys(), vtype=GRB.CONTINUOUS, name="sales")
purchase = m.addVars(purchase_cost.keys(), vtype=GRB.CONTINUOUS, name="purchases")

# objective function
revenue = gp.quicksum(sales[crop] * selling_price[crop] for crop in selling_price)
planting_cost_total = gp.quicksum(planting[crop] * planting_cost[crop] for crop in planting_cost)
purchase_cost_total = gp.quicksum(purchase[crop] * purchase_cost[crop] for crop in purchase_cost)
m.setObjective(revenue - planting_cost_total - purchase_cost_total, GRB.MAXIMIZE)

# constraints
m.addConstr(expected_yields["wheat"] * planting["wheat"] + purchase["wheat"] - sales["wheat"] >= feed_requirements["wheat"], "Wheat Balance")
m.addConstr(expected_yields["corn"] * planting["corn"] + purchase["corn"] - sales["corn"] >= feed_requirements["corn"], "Corn Balance")
m.addConstr(expected_yields["beets"] * planting["beets"] - sales["beets_high"] - sales["beets_low"] >= 0, "Beet Balance")
m.addConstr(sales["beets_high"] <= beet_sale_limit, "Beets Limit")
m.addConstr(planting.sum() <= land_available, "Land Balance")

# solve the model
m.optimize()

# display the results
print(f"\nNet profit: {m.objVal:.0f}")
for crop in planting:
    print(f"  plant {planting[crop].x:.0f} acres of {crop}.")

Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 5 rows, 9 columns and 13 nonzeros
Model fingerprint: 0x0c7f5310
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+01, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 6e+03]
Presolve removed 2 rows and 2 columns
Presolve time: 0.01s
Presolved: 3 rows, 7 columns, 9 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.1200000e+33   2.000000e+30   5.120000e+03      0s
       4    1.1860000e+05   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.04 seconds (0.00 work units)
Optimal objective  1.186000000e+05

Net profit: 118600
  plant 120 acres of wheat.
  plant 80 acr

Then, we evaluate the profit for each scenario and calculate the average.

In [ ]:
def resouceFunction(scenario, planting):
    # create Gurobi model
    m = gp.Model("Resource Function")
    # decision variables
    sales = m.addVars(selling_price.keys(), vtype=GRB.CONTINUOUS, name="sales")
    purchase = m.addVars(purchase_cost.keys(), vtype=GRB.CONTINUOUS, name="purchases")
    # objective function
    revenue = gp.quicksum(sales[crop] * selling_price[crop] for crop in selling_price)
    planting_cost_total = gp.quicksum(planting[crop].x * planting_cost[crop] for crop in planting_cost)
    purchase_cost_total = gp.quicksum(purchase[crop] * purchase_cost[crop] for crop in purchase_cost)
    m.setObjective(revenue - planting_cost_total - purchase_cost_total, GRB.MAXIMIZE)
    # constraints
    m.addConstr(yields[scenario]["wheat"] * planting["wheat"].x + purchase["wheat"] - sales["wheat"] >= feed_requirements["wheat"], "Wheat Balance")
    m.addConstr(yields[scenario]["corn"] * planting["corn"].x + purchase["corn"] - sales["corn"] >= feed_requirements["corn"], "Corn Balance")
    m.addConstr(yields[scenario]["beets"] * planting["beets"].x - sales["beets_high"] - sales["beets_low"] >= 0, "Beet Balance")
    m.addConstr(sales["beets_high"] <= beet_sale_limit, "Beets Limit")
    return m

# solve for each senaraio
# scenario 1
q1 = resouceFunction(1, planting)
q1.optimize()
obj1 = q1.objVal
# scenario 2
q2 = resouceFunction(2, planting)
q2.optimize()
obj2 = q2.objVal
# scenario 3
q3 = resouceFunction(3, planting)
q3.optimize()
obj3 = q3.objVal

# result
print(f"\nProfit for Scenario 1: {obj1:.0f}")
print(f"Profit for Scenario 2: {obj2:.0f}")
print(f"Profit for Scenario 3: {obj3:.0f}")
expected_profit = probabilities[1] * obj1 + probabilities[2] * obj2 + probabilities[3] * obj3
print(f"Expected Profit: {expected_profit:.0f}")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 4 rows, 6 columns and 7 nonzeros
Model fingerprint: 0x12edb664
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 6e+03]
Presolve removed 4 rows and 6 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.1860000e+05   0.000000e+00   1.000000e+01      0s
Extra simplex iterations after uncrush: 1
       1    1.1860000e+05   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.186000000e+05
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: 

#### Extensive Form

In each scenario, we can either buy crops to meet the feed requirement or sell excess production. We'll formulate a extensive form to account for these scenarios.

Question:

- Which decision variables are first-stage? Which one should be decided now and here?
- Which decision variables are second-stage? Which one can be decided in the future?

In [ ]:
# create Gurobi model
m = gp.Model("Extensive Form")

# TODO: decision variables
planting = m.addVars(planting_cost.keys(), vtype=GRB.CONTINUOUS, name="plants")
purchase = m.addVars(scenarios, purchase_cost.keys(), vtype=GRB.CONTINUOUS,
name="purchases")
sales = m.addVars(scenarios, selling_price.keys(), vtype=GRB.CONTINUOUS,
name="sales")


# TODO: objective function
revenue = gp.quicksum(probabilities[sc] * sales[sc, crop] * selling_price[crop] for crop in selling_price for sc in scenarios)
planting_cost_total = gp.quicksum(planting[crop] * planting_cost[crop] for crop in planting_cost)
purchase_cost_total = gp.quicksum(probabilities[sc] * purchase[sc, crop] * purchase_cost[crop] for crop in purchase_cost for sc in scenarios)
m.setObjective(revenue - planting_cost_total - purchase_cost_total, GRB.MAXIMIZE)

# TODO: constraints
m.addConstrs((yields[sc]["wheat"] * planting["wheat"] + purchase[sc, "wheat"] - sales[sc, "wheat"] >= feed_requirements["wheat"] for sc in scenarios), "Wheat Balance")
m.addConstrs((yields[sc]["corn"] * planting["corn"] + purchase[sc, "corn"] - sales[sc, "corn"] >= feed_requirements["corn"] for sc in scenarios), "Corn Balance")
m.addConstrs((yields[sc]["beets"] * planting["beets"] - sales[sc, "beets_high"] - sales[sc, "beets_low"] >= 0 for sc in scenarios), "Beet Balance")
m.addConstrs((sales[sc, "beets_high"] <= beet_sale_limit for sc in scenarios), "Beets Limit")
m.addConstr(planting.sum() <= land_available, "Land Balance")

# solve the model
m.optimize()

# display the results
print(f"\nNet profit: {m.objVal:.0f}")
for crop in planting:
    print(f"  plant {planting[crop].x:.0f} acres of {crop}.")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 13 rows, 21 columns and 33 nonzeros
Model fingerprint: 0x5c296771
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [3e+00, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 6e+03]
Presolve removed 4 rows and 1 columns
Presolve time: 0.01s
Presolved: 9 rows, 20 columns, 30 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.7200000e+33   8.000000e+30   4.720000e+03      0s
      14    1.0839000e+05   0.000000e+00   0.000000e+00      0s

Solved in 14 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.083900000e+05

Net profit: 108390
  plant 170 acres of wheat.
  plant 80 acres of corn.
  plant 250 acres of beets.


{(1, 'wheat'): <gurobi.Var purchases[1,wheat] (value 0.0)>,
 (1, 'corn'): <gurobi.Var purchases[1,corn] (value 0.0)>,
 (2, 'wheat'): <gurobi.Var purchases[2,wheat] (value 0.0)>,
 (2, 'corn'): <gurobi.Var purchases[2,corn] (value 0.0)>,
 (3, 'wheat'): <gurobi.Var purchases[3,wheat] (value 0.0)>,
 (3, 'corn'): <gurobi.Var purchases[3,corn] (value 47.99999999999997)>}

Question:

- How much of an improvement is stochastic programming over LP with expectations?

#### Bender's Decomposition

To implement Bender’s Decomposition based on the explanation you've provided, we can break down the process into two main components: the master problem and the subproblem for each scenario.

When we have a maximization problem, we can turn it into a minimization problem by changing the sign of the objective function. Instead of maximizing $f(x)$, we minimize $−f(x)$.

##### Master Problem

We will create the master problem in Gurobi, and for each iteration, solve the subproblems to add new constraints.

In [ ]:
# master problem creation
master = gp.Model("Benders Master")

# turn off log
master.Params.outputFlag = 0

# TODO: first-stage decision variables


# recourse approximation variables
η = master.addVars(scenarios, vtype=GRB.CONTINUOUS, lb=-1e9, name="η")

# TODO: objective function for the master problem
master.setObjective

# TODO: constraint
master.addConstr

##### Subproblems

For each scenario, we solve a subproblem to determine the second-stage decisions and obtain the dual variables. If necessary, generate a new cut and add it to the master problem.

we typically solve the primal form of the subproblem to make things more intuitive and practical. Whether we solve the primal or the dual subproblem, we can always extract the dual variables.


In [ ]:
# subproblem
def subproblem(scenario, planting):
    # create Gurobi model
    subprob = gp.Model("Subproblem")
    # turn off log
    subprob.Params.outputFlag = 0
    # TODO: decision variables

    # TODO: objective function
    subprob.setObjective
    # TODO: constraints
    subprob.addConstr
    return subprob

##### Iterative Approach

The process iteratively adds new Bender’s cuts to the master problem, updating it with the information from the subproblems until convergence is achieved.

In [ ]:
# init cnt
iter = 0
# Bender's decomposition loop
while True:
    # count
    iter += 1
    # initialize a flag to check if any cuts were added
    cuts_added = False
    # solve the master
    master.optimize()
    print(f"Iteration {iter}, Objective Value: {-master.objVal:.0F}")
    for crop in planting:
        print(f"  plant {planting[crop].x:.0f} acres of {crop}.")
    print("\n")
    # for each scenario
    for sc in scenarios:
        # solve subproblem for the current scenario
        subprob = subproblem(sc, planting)
        subprob.optimize()
        # get the objective value of the subproblem
        Q_value = subprob.objVal
        # check adding a cut
        if η[sc].x < Q_value - 1e-6:
            # extract dual variables from subproblem constraints
            dual_wheat = subprob.getConstrByName("Wheat Balance").pi
            dual_corn = subprob.getConstrByName("Corn Balance").pi
            dual_beets = subprob.getConstrByName("Beet Balance").pi
            dual_beets_limit = subprob.getConstrByName("Beets Limit").pi
            # TODO: formulate the cut using dual values and add it to the master problem
            cut_expr =
            master.addConstr(η[sc] >= cut_expr, f"BendersCut {sc} at Iter {iter}")
            cuts_added = True
    # if no new cuts are added, stop the loop
    if not cuts_added:
        print("Convergence reached. No more cuts needed.")
        break

In [ ]:
# display the results
print(f"\nNet profit: {-master.objVal:.0f}")
for crop in planting:
    print(f"  plant {planting[crop].x:.0f} acres of {crop}.")

Question:

- Why do we set the lower bound of $\eta$ to $-10^9$?